# LinkedIn Auto Post (APIs Linkedin + OpenAI)
## Token Management, Folder Reading, and OpenAI Integration

In [ ]:
# Cell 1: Imports and Configuration
import os
import json
import requests
from pathlib import Path
from datetime import datetime, timedelta
from openai import OpenAI
import mimetypes
import time

# API Keys
OPENAI_API_KEY = 'your api here'
LINKEDIN_TOKEN = 'your token here'
CLIENT_ID = 'your client id here'
CLIENT_SECRET = 'your secret here'

# Token metadata file
TOKEN_META_FILE = 'linkedin_token_meta.json'

openai_client = OpenAI(api_key=OPENAI_API_KEY)
print("✓ Configuration loaded")

In [ ]:
# Cell 2: Token Expiration Management
def save_token_metadata(token, expires_in=5183999):
    """Save token with creation date"""
    metadata = {
        'token': token,
        'created_at': datetime.now().isoformat(),
        'expires_in_seconds': expires_in
    }
    with open(TOKEN_META_FILE, 'w') as f:
        json.dump(metadata, f)

def check_token_expiration():
    """Check days until token expires"""
    if not os.path.exists(TOKEN_META_FILE):
        save_token_metadata(LINKEDIN_TOKEN)
        return "⚠️  Token metadata created. Expires in ~60 days"
    
    with open(TOKEN_META_FILE, 'r') as f:
        metadata = json.load(f)
    
    created = datetime.fromisoformat(metadata['created_at'])
    expires_in = metadata['expires_in_seconds']
    expiry_date = created + timedelta(seconds=expires_in)
    days_left = (expiry_date - datetime.now()).days
    
    if days_left < 0:
        return f"❌ TOKEN EXPIRED {abs(days_left)} days ago! Run 'Linkedin API Token.ipynb'"
    elif days_left < 7:
        return f"⚠️  Token expires in {days_left} days!"
    else:
        return f"✓ Token valid ({days_left} days remaining)"

# Check token status
print(check_token_expiration())

In [ ]:
# Cell 3: Folder Content Reader
def read_folder_contents(folder_path):
    """Read all files from folder"""
    folder = Path(folder_path)
    if not folder.exists():
        raise FileNotFoundError(f"Folder not found: {folder_path}")
    
    contents = {
        'readme': None,
        'images': [],
        'videos': [],
        'other': []
    }
    
    # Find README
    readme_files = list(folder.glob('README.*')) + list(folder.glob('readme.*'))
    if readme_files:
        with open(readme_files[0], 'r', encoding='utf-8') as f:
            contents['readme'] = f.read()
    
    # Categorize media files
    for file in folder.iterdir():
        if file.is_file():
            mime_type, _ = mimetypes.guess_type(str(file))
            if mime_type:
                if mime_type.startswith('image/'):
                    contents['images'].append(str(file))
                elif mime_type.startswith('video/'):
                    contents['videos'].append(str(file))
    
    return contents

# Test function
# folder_contents = read_folder_contents('/path/to/your/folder')
# print(f"Found: {len(folder_contents['images'])} images, {len(folder_contents['videos'])} videos")

In [ ]:
# Cell 4: OpenAI Content Analysis
def generate_post_content(readme_text, num_images=0, num_videos=0):
    """Generate technical caption and hashtags using OpenAI"""
    prompt = f"""Analyze this technical project and create a LinkedIn post in a professional, technical tone.

Project Documentation:
{readme_text}

Media: {num_images} images, {num_videos} videos

Requirements:
1. Write a technical, objective caption (NO excessive emojis, NO emotional language)
2. Focus on: technical implementation, technologies used, challenges solved, architecture decisions
3. Use bullet points for clarity
4. Start with a brief technical overview (1-2 sentences)
5. Explain HOW things were built and WHY (technical reasoning)
6. Mention specific technologies, APIs, algorithms, or frameworks
7. Include 6-8 relevant technical hashtags

Format EXACTLY like this example:

CAPTION:
💡 How I [brief technical objective].

I started this project by [initial challenge/problem].

To solve this, I built [technical solution]:
- [Technical detail 1]
- [Technical detail 2]
- [Technical detail 3]

I used [technologies/tools/specs] to achieve [technical outcome].

The final implementation [results and technical achievement].

✨ [Brief conclusion about technical learning or impact].

HASHTAGS:
#TechStack #API #Automation #Python #MachineLearning #DevOps #TechnicalHashtag #RelevantTech

Now create the post:
"""
    
    response = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.5,  # Lower temperature for more technical/precise output
        max_tokens=400
    )
    
    content = response.choices[0].message.content
    
    # Parse response
    parts = content.split('HASHTAGS:')
    caption = parts[0].replace('CAPTION:', '').strip()
    hashtags = parts[1].strip() if len(parts) > 1 else ''
    
    return caption, hashtags

print("✓ Technical post generation function loaded")

In [ ]:
# Cell 5: Get LinkedIn User URN
def get_user_urn():
    """Get LinkedIn user URN for posting"""
    headers = {'Authorization': f'Bearer {LINKEDIN_TOKEN}'}
    response = requests.get('https://api.linkedin.com/v2/userinfo', headers=headers)
    
    if response.status_code == 200:
        user_data = response.json()
        return user_data['sub']
    else:
        raise Exception(f"Failed to get user info: {response.status_code} - {response.text}")

# Get and store URN
USER_URN = get_user_urn()
print(f"✓ User URN obtained: {USER_URN}")

In [ ]:
# Cell 6: Export Data
def prepare_data_for_part2(folder_path):
    """Prepare all data needed for Part 2"""
    print("Reading folder contents...")
    contents = read_folder_contents(folder_path)
    
    print("Generating post content with OpenAI...")
    caption, hashtags = generate_post_content(
        contents['readme'] or "No README found",
        len(contents['images']),
        len(contents['videos'])
    )
    
    post_data = {
        'caption': caption,
        'hashtags': hashtags,
        'full_text': f"{caption}\n\n{hashtags}",
        'images': contents['images'],
        'videos': contents['videos'],
        'user_urn': USER_URN,
        'token': LINKEDIN_TOKEN
    }
    
    # Save to JSON for Part 2
    with open('post_data.json', 'w', encoding='utf-8') as f:
        json.dump(post_data, f, indent=2, ensure_ascii=False)
    
    print("\n" + "="*50)
    print("POST PREVIEW:")
    print("="*50)
    print(post_data['full_text'])
    print("="*50)
    print(f"\nMedia: {len(post_data['images'])} images, {len(post_data['videos'])} videos")
    print("\n✓ Data saved to 'post_data.json'")
    print("\n📋 Ready for Part 2!")
    
    return post_data

# Example usage:
# folder_path = input("Enter folder path: ")
# post_data = prepare_data_for_part2(folder_path)

In [ ]:
# Cell 7: Files path
print("📁 Enter the full path to the folder with original images:")
folder_path = input("Path: ").strip()

# Remover aspas se o usuário colar o caminho com aspas
folder_path = folder_path.strip('"').strip("'")

# Preparar dados e criar post_data.json
post_data = prepare_data_for_part2(folder_path)

In [ ]:
# Cell 8: Imports and Load Data
# Load data from Part 1
with open('post_data.json', 'r', encoding='utf-8') as f:
    POST_DATA = json.load(f)

TOKEN = POST_DATA['token']
USER_URN = POST_DATA['user_urn']

print("✓ Post data loaded")
print(f"Images: {len(POST_DATA['images'])}")
print(f"Videos: {len(POST_DATA['videos'])}")
print(f"\nCaption Preview:\n{POST_DATA['caption'][:100]}...")

In [ ]:
# Cell 9: Image Upload Function
def upload_image(image_path):
    """Upload image to LinkedIn using 3-step process"""
    headers = {
        'Authorization': f'Bearer {TOKEN}',
        'Content-Type': 'application/json'
    }
    
    # Fix: Ensure USER_URN has correct format
    if not USER_URN.startswith('urn:li:person:'):
        owner_urn = f'urn:li:person:{USER_URN}'
    else:
        owner_urn = USER_URN
    
    # Step 1: Register upload
    register_payload = {
        "registerUploadRequest": {
            "recipes": ["urn:li:digitalmediaRecipe:feedshare-image"],
            "owner": owner_urn,  # ← CORRIGIDO
            "serviceRelationships": [{
                "relationshipType": "OWNER",
                "identifier": "urn:li:userGeneratedContent"
            }]
        }
    }
    
    print(f"  Registering upload for {Path(image_path).name}...")
    print(f"  Using owner URN: {owner_urn}")  # Debug
    
    response = requests.post(
        'https://api.linkedin.com/v2/assets?action=registerUpload',
        headers=headers,
        json=register_payload
    )
    
    if response.status_code != 200:
        print(f"  Error details: {response.json()}")  # Debug
        raise Exception(f"Registration failed: {response.status_code} - {response.text}")
    
    register_data = response.json()
    upload_url = register_data['value']['uploadMechanism']['com.linkedin.digitalmedia.uploading.MediaUploadHttpRequest']['uploadUrl']
    asset_urn = register_data['value']['asset']
    
    # Step 2: Upload binary data
    print(f"  Uploading binary data...")
    with open(image_path, 'rb') as f:
        image_data = f.read()
    
    upload_headers = {'Authorization': f'Bearer {TOKEN}'}
    upload_response = requests.put(upload_url, headers=upload_headers, data=image_data)
    
    if upload_response.status_code not in [200, 201]:
        raise Exception(f"Upload failed: {upload_response.status_code}")
    
    print(f"  ✓ Uploaded successfully")
    return asset_urn

print("✓ Image upload function ready")

In [ ]:
# Cell 10: Video Upload Function
def upload_video(video_path):
    """Upload video to LinkedIn using 3-step process"""
    headers = {
        'Authorization': f'Bearer {TOKEN}',
        'Content-Type': 'application/json'
    }
    
    file_size = os.path.getsize(video_path)
    
    # Step 1: Initialize video upload
    init_payload = {
        "initializeUploadRequest": {
            "owner": USER_URN,
            "fileSizeBytes": file_size,
            "uploadCaptions": False,
            "uploadThumbnail": False
        }
    }
    
    print(f"  Initializing video upload for {Path(video_path).name}...")
    response = requests.post(
        'https://api.linkedin.com/v2/videos?action=initializeUpload',
        headers=headers,
        json=init_payload
    )
    
    if response.status_code != 200:
        raise Exception(f"Video init failed: {response.status_code} - {response.text}")
    
    init_data = response.json()
    video_urn = init_data['value']['video']
    upload_instructions = init_data['value']['uploadInstructions']
    
    # Step 2: Upload video in chunks
    print(f"  Uploading video data...")
    with open(video_path, 'rb') as f:
        for instruction in upload_instructions:
            f.seek(instruction['firstByte'])
            chunk = f.read(instruction['lastByte'] - instruction['firstByte'] + 1)
            
            upload_response = requests.put(
                instruction['uploadUrl'],
                headers={'Authorization': f'Bearer {TOKEN}'},
                data=chunk
            )
            
            if upload_response.status_code not in [200, 201]:
                raise Exception(f"Video chunk upload failed: {upload_response.status_code}")
    
    # Step 3: Finalize upload
    finalize_payload = {
        "finalizeUploadRequest": {
            "video": video_urn,
            "uploadToken": "",
            "uploadedPartIds": []
        }
    }
    
    finalize_response = requests.post(
        'https://api.linkedin.com/v2/videos?action=finalizeUpload',
        headers=headers,
        json=finalize_payload
    )
    
    print(f"  ✓ Video uploaded successfully")
    return video_urn

print("✓ Video upload function ready")

In [ ]:
# Cell 11: Upload All Media
def upload_all_media():
    """Upload all images and videos, return asset URNs"""
    asset_urns = []
    
    # Upload images
    if POST_DATA['images']:
        print(f"\nUploading {len(POST_DATA['images'])} image(s)...")
        for image_path in POST_DATA['images']:
            try:
                urn = upload_image(image_path)
                asset_urns.append(urn)
                time.sleep(1)  # Rate limiting
            except Exception as e:
                print(f"  ❌ Failed to upload {image_path}: {e}")
    
    # Upload videos
    if POST_DATA['videos']:
        print(f"\nUploading {len(POST_DATA['videos'])} video(s)...")
        for video_path in POST_DATA['videos']:
            try:
                urn = upload_video(video_path)
                asset_urns.append(urn)
                time.sleep(2)  # Rate limiting
            except Exception as e:
                print(f"  ❌ Failed to upload {video_path}: {e}")
    
    return asset_urns

# Upload media
print("Starting media upload...")
UPLOADED_ASSETS = upload_all_media()
print(f"\n✓ Total assets uploaded: {len(UPLOADED_ASSETS)}")

In [ ]:
# Cell 12: Create LinkedIn Post
def create_linkedin_post(text, asset_urns):
    """Create LinkedIn post with text and media"""
    headers = {
        'Authorization': f'Bearer {TOKEN}',
        'Content-Type': 'application/json',
        'X-Restli-Protocol-Version': '2.0.0'
    }
    
    # Fix: Ensure USER_URN has correct format for author field
    if not USER_URN.startswith('urn:li:person:'):
        author_urn = f'urn:li:person:{USER_URN}'
    else:
        author_urn = USER_URN
    
    print(f"  Using author URN: {author_urn}")  # Debug
    
    # Determine media category
    has_videos = any('video' in urn.lower() for urn in asset_urns)
    media_category = "VIDEO" if has_videos else "IMAGE"
    
    # Build media objects
    media_objects = []
    for urn in asset_urns:
        media_objects.append({
            "status": "READY",
            "media": urn
        })
    
    # Create post payload
    post_payload = {
        "author": author_urn,  # ← CORRIGIDO
        "lifecycleState": "PUBLISHED",
        "specificContent": {
            "com.linkedin.ugc.ShareContent": {
                "shareCommentary": {
                    "text": text
                },
                "shareMediaCategory": media_category,
                "media": media_objects
            }
        },
        "visibility": {
            "com.linkedin.ugc.MemberNetworkVisibility": "PUBLIC"
        }
    }
    
    print("\nCreating LinkedIn post...")
    print(f"  Post payload preview:")
    print(f"  - Author: {author_urn}")
    print(f"  - Media category: {media_category}")
    print(f"  - Assets: {len(asset_urns)}")
    
    response = requests.post(
        'https://api.linkedin.com/v2/ugcPosts',
        headers=headers,
        json=post_payload
    )
    
    if response.status_code == 201:
        post_id = response.headers.get('X-RestLi-Id', '')
        return post_id, response.json()
    else:
        print(f"  Error response: {response.json()}")  # Debug
        raise Exception(f"Post creation failed: {response.status_code} - {response.text}")

print("✓ Post creation function ready")

In [ ]:
# Cell 13: Publish Post
try:
    print("="*60)
    print("PUBLISHING POST")
    print("="*60)
    
    post_id, post_response = create_linkedin_post(
        POST_DATA['full_text'],
        UPLOADED_ASSETS
    )
    
    print("\n" + "="*60)
    print("✓ POST PUBLISHED SUCCESSFULLY!")
    print("="*60)
    print(f"\nPost ID: {post_id}")
    print(f"\nView your post on LinkedIn:")
    print(f"https://www.linkedin.com/feed/update/{post_id}")
    print("\n" + "="*60)
    
    # Save post metadata
    post_metadata = {
        'post_id': post_id,
        'post_url': f"https://www.linkedin.com/feed/update/{post_id}",
        'text': POST_DATA['full_text'],
        'assets_uploaded': len(UPLOADED_ASSETS),
        'timestamp': time.strftime('%Y-%m-%d %H:%M:%S')
    }
    
    with open('post_result.json', 'w', encoding='utf-8') as f:
        json.dump(post_metadata, f, indent=2, ensure_ascii=False)
    
    print("\n✓ Post metadata saved to 'post_result.json'")
    
except Exception as e:
    print(f"\n❌ ERROR: {e}")
    print("\nPlease check:")
    print("- Token is still valid")
    print("- Media files are accessible")
    print("- LinkedIn API permissions are correct")